<a href="https://colab.research.google.com/github/gagandeepsinghkukreja/ABM_V2G_Grid/blob/main/EPREL_API_sync.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Run on chrome where you have
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Link to the Atlassian helper document: https://webgate.ec.europa.eu/fpfis/wikis/pages/viewpage.action?pageId=1961560719

In [ ]:
response = requests.get(paginated_url, headers=headers)
print(response.json())  # Print the full response for debugging



{'size': 6207, 'offset': 0, 'hits': [{'energyConsELEFL': 1.66, 'implementingAct': 'EU_392_2012', 'modelIdentifier': 'RC90V9AV3Q', 'builtIn': False, 'weightedEnergyAnnualGas': None, 'organisation': {'organisationTitle': 'LG Electronics European Shared Service Center B.V.', 'closeDate': None, 'firstName': None, 'lastName': None, 'organisationName': 'LG Electronics European Shared Service Center B.V.', 'website': None, 'closeStatus': None, 'isClosed': False}, 'firstPublicationDateTS': 1563401031, 'orgVerificationStatus': 'VERIFIED', 'type': 'CONDENSER', 'onMarketStartDate': [2019, 7, 1], 'weightedEnergyAnnual': 194, 'energyClassImageWithScale': 'APPP-Left-DarkGreen-WithAPPPDScale.svg', 'weightedProgrammeTime': 169, 'averageCondensationEfficiencyFL': 91, 'trademarkOwner': None, 'onMarketEndDateTS': 4476643200, 'energyConsGASPL': None, 'lastVersion': False, 'formType': 'TYPE_1', 'energyConsGASFL': None, 'onMarketStartDateTS': 1561939200, 'programmeTimePL': 124, 'contactId': None, 'energyCon

In [ ]:
import requests
import pandas as pd
import time


# Define the base URL and headers for Dishwashers

base_url = "https://eprel.ec.europa.eu/api/products/dishwashers2019"
headers = {
    "x-api-key": ""  # Replace with your valid API key
}

# Initialize variables for pagination
all_data = []  # To store all rows of data
page = 1       # Start from page 1
entries_per_request = 25  # Entries fetched per request

# Track unique entries
seen_ids = set()  # To track unique entries by `eprelRegistrationNumber`

while True:
    # Construct paginated URL with query parameters
    paginated_url = (
        f"{base_url}?"
        f"_page={page}&_limit={entries_per_request}"
        f"&dimensionHeightMin=1&dimensionHeightMax=999"
        f"&dimensionWidthMin=1&dimensionWidthMax=999"
        f"&dimensionDepthMin=1&dimensionDepthMax=999"
        f"&ratedCapacityMin=1&ratedCapacityMax=100"
        f"&energyCons100Min=0&energyCons100Max=999"
        f"&noiseMin=1&noiseMax=99"
        f"&sort0=energyClass&order0=DESC"
        f"&sort1=energyCons100&order1=ASC"
    )

    # Log the current request URL
    print(f"Requesting: {paginated_url}")

    # Make GET request to fetch data
    response = requests.get(paginated_url, headers=headers)

    if response.status_code == 200:
        # Parse JSON response
        response_json = response.json()

        # Extract product data from 'hits'
        data = response_json.get("hits", [])

        # Check if no more data is available
        if not data:
            print("No more data available.")
            break

        # Filter out duplicates by checking unique `eprelRegistrationNumber`
        unique_data = [
            item for item in data
            if item.get("eprelRegistrationNumber") not in seen_ids
        ]

        # Update seen IDs
        seen_ids.update(item.get("eprelRegistrationNumber") for item in unique_data)

        # Append only unique data to the list
        all_data.extend(unique_data)

        print(f"Fetched {len(unique_data)} unique records. Page: {page}")

        # Increment page for the next set of results
        page += 1

        # Add a delay to comply with rate limits (5 requests per second)
        time.sleep(0.2)  # 0.2 seconds = 5 requests per second

    elif response.status_code == 429:
        # Handle rate-limiting error by checking Retry-After header or waiting a default time
        retry_after = int(response.headers.get("Retry-After", 60))  # Default to 60 seconds if not provided
        print(f"Rate limit reached. Retrying after {retry_after} seconds...")
        time.sleep(retry_after)

    else:
        print(f"GET request failed with status code {response.status_code}.")
        break

# Convert aggregated data into Pandas DataFrame
df = pd.DataFrame(all_data)

# Save DataFrame to CSV file
csv_filename = "dishwashers_data.csv"
df.to_csv(csv_filename, index=False)

print(f"All data successfully saved to {csv_filename}")


Requesting: https://eprel.ec.europa.eu/api/products/dishwashers2019?_page=1&_limit=25&dimensionHeightMin=1&dimensionHeightMax=999&dimensionWidthMin=1&dimensionWidthMax=999&dimensionDepthMin=1&dimensionDepthMax=999&ratedCapacityMin=1&ratedCapacityMax=100&energyCons100Min=0&energyCons100Max=999&noiseMin=1&noiseMax=99&sort0=energyClass&order0=DESC&sort1=energyCons100&order1=ASC
Fetched 25 unique records. Page: 1
Requesting: https://eprel.ec.europa.eu/api/products/dishwashers2019?_page=2&_limit=25&dimensionHeightMin=1&dimensionHeightMax=999&dimensionWidthMin=1&dimensionWidthMax=999&dimensionDepthMin=1&dimensionDepthMax=999&ratedCapacityMin=1&ratedCapacityMax=100&energyCons100Min=0&energyCons100Max=999&noiseMin=1&noiseMax=99&sort0=energyClass&order0=DESC&sort1=energyCons100&order1=ASC
Fetched 25 unique records. Page: 2
Requesting: https://eprel.ec.europa.eu/api/products/dishwashers2019?_page=3&_limit=25&dimensionHeightMin=1&dimensionHeightMax=999&dimensionWidthMin=1&dimensionWidthMax=999&di

In [ ]:
import requests
import pandas as pd
import time

# Define the base URL and headers for domestic ovens
base_url = "https://eprel.ec.europa.eu/api/products/ovens"
headers = {
    "x-api-key": "8uDFzzII9X9mzRJ4OLZyk5qZtBKX9JZu5I6szmmN"  # Replace with your valid API key
}

# Initialize variables for pagination
all_data = []  # To store all rows of data
page = 1       # Start from page 1
entries_per_request = 50  # Increase batch size for fewer requests

# Track unique entries
seen_ids = set()  # To track unique entries by `eprelRegistrationNumber`

while True:
    # Construct paginated URL with query parameters
    paginated_url = (
        f"{base_url}?"
        f"_page={page}&_limit={entries_per_request}"
    )

    # Log the current request URL
    print(f"Requesting: {paginated_url}")

    # Make GET request to fetch data
    response = requests.get(paginated_url, headers=headers)

    if response.status_code == 200:
        # Parse JSON response
        response_json = response.json()

        # Extract product data from 'hits'
        data = response_json.get("hits", [])

        # Check if no more data is available
        if not data:
            print("No more data available.")
            break

        # Filter out duplicates by checking unique `eprelRegistrationNumber`
        unique_data = [
            item for item in data
            if item.get("eprelRegistrationNumber") not in seen_ids
        ]

        # Update seen IDs
        seen_ids.update(item.get("eprelRegistrationNumber") for item in unique_data)

        # Append only unique data to the list
        all_data.extend(unique_data)

        print(f"Fetched {len(unique_data)} unique records. Page: {page}")

        # Increment page for the next set of results
        page += 1

        # Add a delay to comply with rate limits (5 requests per second)
        time.sleep(0.5)  # Increase delay to ensure compliance

    elif response.status_code == 429:
        # Handle rate-limiting error by checking Retry-After header or waiting a default time
        retry_after = int(response.headers.get("Retry-After", 60))  # Default to 60 seconds if not provided
        print(f"Rate limit reached. Retrying after {retry_after} seconds...")
        time.sleep(retry_after)

    else:
        print(f"GET request failed with status code {response.status_code}.")
        break

# Convert aggregated data into Pandas DataFrame
df = pd.DataFrame(all_data)

# Save DataFrame to CSV file
csv_filename = "domestic_ovens_data.csv"
df.to_csv(csv_filename, index=False)

print(f"All data successfully saved to {csv_filename}")


Requesting: https://eprel.ec.europa.eu/api/products/ovens?_page=1&_limit=50
Fetched 50 unique records. Page: 1
Requesting: https://eprel.ec.europa.eu/api/products/ovens?_page=2&_limit=50
Fetched 50 unique records. Page: 2
Requesting: https://eprel.ec.europa.eu/api/products/ovens?_page=3&_limit=50
Fetched 50 unique records. Page: 3
Requesting: https://eprel.ec.europa.eu/api/products/ovens?_page=4&_limit=50
Fetched 39 unique records. Page: 4
Requesting: https://eprel.ec.europa.eu/api/products/ovens?_page=5&_limit=50
Fetched 50 unique records. Page: 5
Requesting: https://eprel.ec.europa.eu/api/products/ovens?_page=6&_limit=50
Fetched 50 unique records. Page: 6
Requesting: https://eprel.ec.europa.eu/api/products/ovens?_page=7&_limit=50
Fetched 50 unique records. Page: 7
Requesting: https://eprel.ec.europa.eu/api/products/ovens?_page=8&_limit=50
Fetched 50 unique records. Page: 8
Requesting: https://eprel.ec.europa.eu/api/products/ovens?_page=9&_limit=50
Fetched 50 unique records. Page: 9
R

In [ ]:
import requests
import pandas as pd
import time

# Define the base URL and headers for washing machines
base_url = "https://eprel.ec.europa.eu/api/products/washingmachines2019"
headers = {
    "x-api-key": "8uDFzzII9X9mzRJ4OLZyk5qZtBKX9JZu5I6szmmN"  # Replace with your valid API key
}

# Initialize variables for pagination and data collection
all_data = []  # To store all rows of data
seen_ids = set()  # To track unique entries by `eprelRegistrationNumber`
page = 1  # Start from page 1
entries_per_request = 25  # Entries fetched per request

while True:
    # Construct the paginated URL with filtering parameters
    paginated_url = (
        f"{base_url}?_page={page}&_limit={entries_per_request}"
        f"&dimensionHeightMin=1&dimensionHeightMax=999"
        f"&dimensionWidthMin=1&dimensionWidthMax=999"
        f"&dimensionDepthMin=1&dimensionDepthMax=999"
        f"&ratedCapacityMin=0.1&ratedCapacityMax=99.9"
        f"&energyConsMin=0&energyConsMax=999"
        f"&noiseMin=1&noiseMax=99"
        f"&sort0=energyClass&order0=DESC"
        f"&sort1=energyCons&order1=ASC"
    )

    # Make the GET request
    response = requests.get(paginated_url, headers=headers)

    if response.status_code == 200:
        # Parse JSON response
        response_json = response.json()

        # Extract product data from 'hits'
        data = response_json.get("hits", [])

        # Check if no more data is available
        if not data:
            print("No more data available.")
            break

        # Filter out duplicates by checking unique `eprelRegistrationNumber`
        unique_data = [
            item for item in data
            if item.get("eprelRegistrationNumber") not in seen_ids
        ]

        # Update seen IDs
        seen_ids.update(item.get("eprelRegistrationNumber") for item in unique_data)

        # Append only unique data to the list
        all_data.extend(unique_data)

        print(f"Fetched {len(unique_data)} unique records. Page: {page}")

        # Increment page for the next set of results
        page += 1

        # Add a delay to comply with rate limits (5 requests per second)
        time.sleep(0.2)

    elif response.status_code == 429:
        # Handle rate-limiting error by checking Retry-After header or waiting a default time
        retry_after = int(response.headers.get("Retry-After", 60))  # Default to 60 seconds if not provided
        print(f"Rate limit reached. Retrying after {retry_after} seconds...")
        time.sleep(retry_after)

    else:
        print(f"GET request failed with status code {response.status_code}.")
        break

# Convert aggregated data into Pandas DataFrame
df = pd.DataFrame(all_data)

# Save DataFrame to CSV file
csv_filename = "washing_machines_data.csv"
df.to_csv(csv_filename, index=False)

print(f"All data successfully saved to {csv_filename}")


Fetched 25 unique records. Page: 1
Fetched 25 unique records. Page: 2
Fetched 25 unique records. Page: 3
Fetched 25 unique records. Page: 4
Fetched 25 unique records. Page: 5
Fetched 25 unique records. Page: 6
Fetched 25 unique records. Page: 7
Fetched 25 unique records. Page: 8
Fetched 25 unique records. Page: 9
Fetched 25 unique records. Page: 10
Fetched 25 unique records. Page: 11
Fetched 25 unique records. Page: 12
Fetched 25 unique records. Page: 13
Fetched 25 unique records. Page: 14
Fetched 25 unique records. Page: 15
Fetched 25 unique records. Page: 16
Fetched 25 unique records. Page: 17
Fetched 25 unique records. Page: 18
Fetched 25 unique records. Page: 19
Fetched 25 unique records. Page: 20
Fetched 25 unique records. Page: 21
Fetched 25 unique records. Page: 22
Fetched 25 unique records. Page: 23
Fetched 25 unique records. Page: 24
Fetched 25 unique records. Page: 25
Fetched 25 unique records. Page: 26
Fetched 25 unique records. Page: 27
Fetched 25 unique records. Page: 28
F

In [ ]:
import requests
import pandas as pd
import time

# Define the base URL and headers
base_url = "https://eprel.ec.europa.eu/api/products/tumbledriers"
headers = {
    "x-api-key": "8uDFzzII9X9mzRJ4OLZyk5qZtBKX9JZu5I6szmmN"  # Replace with your valid API key
}

# Initialize variables
all_data = []
seen_ids = set()
page = 1
entries_per_request = 100  # Try fetching more entries per request

while True:
    # Construct paginated URL without unnecessary filters
    paginated_url = f"{base_url}?_page={page}&_limit={entries_per_request}"

    print(f"Requesting: {paginated_url}")  # Debugging

    # Make the GET request
    response = requests.get(paginated_url, headers=headers)

    if response.status_code == 200:
        # Parse JSON response
        response_json = response.json()

        # Extract data
        data = response_json.get("hits", [])

        # Log the number of entries received
        print(f"Page {page}: Fetched {len(data)} records.")

        if not data:  # No more data
            break

        # Deduplicate data
        unique_data = [item for item in data if item.get("eprelRegistrationNumber") not in seen_ids]
        seen_ids.update(item.get("eprelRegistrationNumber") for item in unique_data)

        # Append unique data
        all_data.extend(unique_data)

        # Increment page
        page += 1

        # Respect rate limits
        time.sleep(0.2)

    elif response.status_code == 429:  # Rate-limit handling
        retry_after = int(response.headers.get("Retry-After", 60))
        print(f"Rate limit hit. Retrying after {retry_after} seconds...")
        time.sleep(retry_after)

    else:  # Handle other errors
        print(f"Error: Status code {response.status_code}")
        break

# Convert to DataFrame and save
df = pd.DataFrame(all_data)
csv_filename = "tumble_driers_full_data.csv"
df.to_csv(csv_filename, index=False)
print(f"Data saved to {csv_filename}. Total records: {len(df)}")


Requesting: https://eprel.ec.europa.eu/api/products/tumbledriers?_page=1&_limit=100
Page 1: Fetched 100 records.
Requesting: https://eprel.ec.europa.eu/api/products/tumbledriers?_page=2&_limit=100
Page 2: Fetched 100 records.
Requesting: https://eprel.ec.europa.eu/api/products/tumbledriers?_page=3&_limit=100
Page 3: Fetched 100 records.
Requesting: https://eprel.ec.europa.eu/api/products/tumbledriers?_page=4&_limit=100
Page 4: Fetched 100 records.
Requesting: https://eprel.ec.europa.eu/api/products/tumbledriers?_page=5&_limit=100
Page 5: Fetched 100 records.
Requesting: https://eprel.ec.europa.eu/api/products/tumbledriers?_page=6&_limit=100
Page 6: Fetched 100 records.
Requesting: https://eprel.ec.europa.eu/api/products/tumbledriers?_page=7&_limit=100
Page 7: Fetched 100 records.
Requesting: https://eprel.ec.europa.eu/api/products/tumbledriers?_page=8&_limit=100
Page 8: Fetched 100 records.
Requesting: https://eprel.ec.europa.eu/api/products/tumbledriers?_page=9&_limit=100
Page 9: Fetc

In [ ]:
import requests
import pandas as pd
import time

# Define the base URL and headers
base_url = "https://eprel.ec.europa.eu/api/products/refrigeratingappliances2019"
headers = {
    "x-api-key": "8uDFzzII9X9mzRJ4OLZyk5qZtBKX9JZu5I6szmmN"  # Replace with your valid API key
}

# Initialize variables for pagination and data collection
all_data = []
seen_ids = set()
page = 1
entries_per_request = 100  # Attempt to retrieve more records per request

while True:
    # Construct the paginated URL
    paginated_url = f"{base_url}?_page={page}&_limit={entries_per_request}"

    # Log the request URL for debugging
    print(f"Requesting: {paginated_url}")

    # Send the GET request
    response = requests.get(paginated_url, headers=headers)

    if response.status_code == 200:
        # Parse JSON response
        response_json = response.json()

        # Extract records from 'hits'
        data = response_json.get("hits", [])

        # Check if no more data is available
        if not data:
            print("No more data available. Pagination complete.")
            break

        # Deduplicate entries based on `eprelRegistrationNumber`
        unique_data = [
            item for item in data
            if item.get("eprelRegistrationNumber") not in seen_ids
        ]

        # Update the seen IDs
        seen_ids.update(item.get("eprelRegistrationNumber") for item in unique_data)

        # Append unique records to the list
        all_data.extend(unique_data)

        print(f"Page {page}: Fetched {len(unique_data)} unique records.")

        # Increment the page for the next iteration
        page += 1

        # Respect API rate limits (5 requests per second)
        time.sleep(0.2)

    elif response.status_code == 429:  # Rate-limit handling
        retry_after = int(response.headers.get("Retry-After", 60))
        print(f"Rate limit reached. Retrying after {retry_after} seconds...")
        time.sleep(retry_after)

    else:
        print(f"Error: Request failed with status code {response.status_code}.")
        break

# Convert the collected data to a Pandas DataFrame
df = pd.DataFrame(all_data)

# Save the data to a CSV file
csv_filename = "refrigerating_appliances_data.csv"
df.to_csv(csv_filename, index=False)

print(f"Data successfully saved to {csv_filename}. Total records: {len(df)}")


Requesting: https://eprel.ec.europa.eu/api/products/refrigeratingappliances2019?_page=1&_limit=100
Page 1: Fetched 100 unique records.
Requesting: https://eprel.ec.europa.eu/api/products/refrigeratingappliances2019?_page=2&_limit=100
Page 2: Fetched 100 unique records.
Requesting: https://eprel.ec.europa.eu/api/products/refrigeratingappliances2019?_page=3&_limit=100
Page 3: Fetched 100 unique records.
Requesting: https://eprel.ec.europa.eu/api/products/refrigeratingappliances2019?_page=4&_limit=100
Page 4: Fetched 100 unique records.
Requesting: https://eprel.ec.europa.eu/api/products/refrigeratingappliances2019?_page=5&_limit=100
Page 5: Fetched 100 unique records.
Requesting: https://eprel.ec.europa.eu/api/products/refrigeratingappliances2019?_page=6&_limit=100
Page 6: Fetched 100 unique records.
Requesting: https://eprel.ec.europa.eu/api/products/refrigeratingappliances2019?_page=7&_limit=100
Page 7: Fetched 100 unique records.
Requesting: https://eprel.ec.europa.eu/api/products/ref

In [ ]:
import requests
import pandas as pd
import time

# Define the base URL and headers for Dishwashers
base_url = "https://eprel.ec.europa.eu/api/products/dishwashers2019"
headers = {
    "x-api-key": "8uDFzzII9X9mzRJ4OLZyk5qZtBKX9JZu5I6szmmN"  # Replace with your valid API key
}

# Filtering and sorting parameters based on your requirements
params = {
    "_page": 1,
    "_limit": 100,  # Fetch up to 100 records per page
    "dimensionHeightMin": 1,
    "dimensionHeightMax": 999,
    "dimensionWidthMin": 1,
    "dimensionWidthMax": 999,
    "dimensionDepthMin": 1,
    "dimensionDepthMax": 999,
    "ratedCapacityMin": 1,
    "ratedCapacityMax": 100,
    "energyCons100Min": 0,
    "energyCons100Max": 999,
    "noiseMin": 1,
    "noiseMax": 99,
    "sort0": "energyClass",
    "order0": "DESC",
    "sort1": "energyCons100",
    "order1": "ASC"
}

# Initialize variables for pagination and data collection
all_data = []
seen_ids = set()

while True:
    print(f"Fetching page {params['_page']}...")

    # Send the GET request
    response = requests.get(base_url, headers=headers, params=params)

    if response.status_code == 200:
        # Parse JSON response
        response_json = response.json()

        # Extract records from 'hits'
        data = response_json.get("hits", [])

        # Stop if no more data is available
        if not data:
            print("No more data available. Pagination complete.")
            break

        # Deduplicate entries based on `eprelRegistrationNumber`
        unique_data = [
            item for item in data
            if item.get("eprelRegistrationNumber") not in seen_ids
        ]

        # Update the seen IDs
        seen_ids.update(item.get("eprelRegistrationNumber") for item in unique_data)

        # Append unique records to the list
        all_data.extend(unique_data)

        print(f"Page {params['_page']}: Fetched {len(unique_data)} unique records.")

        # Increment the page for the next iteration
        params["_page"] += 1

        # Respect API rate limits (5 requests per second)
        time.sleep(0.2)

    elif response.status_code == 429:  # Rate-limit handling
        retry_after = int(response.headers.get("Retry-After", 60))
        print(f"Rate limit reached. Retrying after {retry_after} seconds...")
        time.sleep(retry_after)

    else:
        print(f"Error: Request failed with status code {response.status_code}.")
        break

# Convert the collected data to a Pandas DataFrame
df = pd.DataFrame(all_data)

# Save the data to a CSV file
csv_filename = "dishwashers_data.csv"
df.to_csv(csv_filename, index=False)

print(f"Data successfully saved to {csv_filename}. Total records: {len(df)}")


Fetching page 1...
Page 1: Fetched 100 unique records.
Fetching page 2...
Page 2: Fetched 100 unique records.
Fetching page 3...
Page 3: Fetched 100 unique records.
Fetching page 4...
Page 4: Fetched 98 unique records.
Fetching page 5...
Page 5: Fetched 100 unique records.
Fetching page 6...
Page 6: Fetched 100 unique records.
Fetching page 7...
Page 7: Fetched 100 unique records.
Fetching page 8...
Page 8: Fetched 100 unique records.
Fetching page 9...
Page 9: Fetched 100 unique records.
Fetching page 10...
Page 10: Fetched 100 unique records.
Fetching page 11...
Page 11: Fetched 100 unique records.
Fetching page 12...
Page 12: Fetched 100 unique records.
Fetching page 13...
Page 13: Fetched 100 unique records.
Fetching page 14...
Page 14: Fetched 100 unique records.
Fetching page 15...
Page 15: Fetched 100 unique records.
Fetching page 16...
Page 16: Fetched 100 unique records.
Fetching page 17...
Page 17: Fetched 100 unique records.
Fetching page 18...
Page 18: Fetched 100 unique re

In [ ]:
import requests
import pandas as pd
import time

# Define the base URL and headers for TV/Electronic Displays
base_url = "https://eprel.ec.europa.eu/api/products/electronicdisplays"
headers = {
    "x-api-key": "8uDFzzII9X9mzRJ4OLZyk5qZtBKX9JZu5I6szmmN"  # Replace with your valid API key
}

# Filtering and sorting parameters
params = {
    "_page": 1,
    "_limit": 100,  # Fetch up to 100 records per page
    "powerOnModeSDRMin": 0.1,
    "powerOnModeSDRMax": 9999.9,
    "diagonalCmMin": 0.1,
    "diagonalCmMax": 999.9,
    "diagonalInchMin": 1,
    "diagonalInchMax": 999,
    "resolutionHorizontalPixelsMin": 1,
    "resolutionHorizontalPixelsMax": 99999,
    "resolutionVerticalPixelsMin": 1,
    "resolutionVerticalPixelsMax": 99999,
    "sort0": "energyClass",
    "order0": "DESC",
    "sort1": "powerOnModeSDR",
    "order1": "ASC"
}

# Initialize variables for pagination and data collection
all_data = []
seen_ids = set()

while True:
    print(f"Fetching page {params['_page']}...")

    # Send the GET request
    response = requests.get(base_url, headers=headers, params=params)

    if response.status_code == 200:
        # Parse JSON response
        response_json = response.json()

        # Extract records from 'hits'
        data = response_json.get("hits", [])

        # Stop if no more data is available
        if not data:
            print("No more data available. Pagination complete.")
            break

        # Deduplicate entries based on `eprelRegistrationNumber`
        unique_data = [
            item for item in data
            if item.get("eprelRegistrationNumber") not in seen_ids
        ]

        # Update the seen IDs
        seen_ids.update(item.get("eprelRegistrationNumber") for item in unique_data)

        # Append unique records to the list
        all_data.extend(unique_data)

        print(f"Page {params['_page']}: Fetched {len(unique_data)} unique records.")

        # Increment the page for the next iteration
        params["_page"] += 1

        # Respect API rate limits (5 requests per second)
        time.sleep(0.2)

    elif response.status_code == 429:  # Rate-limit handling
        retry_after = int(response.headers.get("Retry-After", 60))
        print(f"Rate limit reached. Retrying after {retry_after} seconds...")
        time.sleep(retry_after)

    else:
        print(f"Error: Request failed with status code {response.status_code}.")
        break

# Convert the collected data to a Pandas DataFrame
df = pd.DataFrame(all_data)

# Save the data to a CSV file
csv_filename = "electronic_displays_data.csv"
df.to_csv(csv_filename, index=False)

print(f"Data successfully saved to {csv_filename}. Total records: {len(df)}")


Fetching page 1...
Page 1: Fetched 100 unique records.
Fetching page 2...
Page 2: Fetched 100 unique records.
Fetching page 3...
Page 3: Fetched 100 unique records.
Fetching page 4...
Page 4: Fetched 100 unique records.
Fetching page 5...
Page 5: Fetched 100 unique records.
Fetching page 6...
Page 6: Fetched 100 unique records.
Fetching page 7...
Page 7: Fetched 100 unique records.
Fetching page 8...
Page 8: Fetched 100 unique records.
Fetching page 9...
Page 9: Fetched 100 unique records.
Fetching page 10...
Page 10: Fetched 100 unique records.
Fetching page 11...
Page 11: Fetched 100 unique records.
Fetching page 12...
Page 12: Fetched 100 unique records.
Fetching page 13...
Page 13: Fetched 100 unique records.
Fetching page 14...
Page 14: Fetched 100 unique records.
Fetching page 15...
Page 15: Fetched 100 unique records.
Fetching page 16...
Page 16: Fetched 100 unique records.
Fetching page 17...
Page 17: Fetched 100 unique records.
Fetching page 18...
Page 18: Fetched 100 unique r

In [ ]:
import requests
import pandas as pd
import time

# Define the base URL and headers for Tumble Driers
base_url = "https://eprel.ec.europa.eu/api/products/tumbledriers"
headers = {
    "x-api-key": "8uDFzzII9X9mzRJ4OLZyk5qZtBKX9JZu5I6szmmN"  # Replace with your valid API key
}

# Filtering and sorting parameters
params = {
    "_page": 1,
    "_limit": 100,  # Fetch up to 100 records per page
    "powerOnModeSDRMin": 0.1,
    "powerOnModeSDRMax": 9999.9,
    "diagonalCmMin": 0.1,
    "diagonalCmMax": 999.9,
    "diagonalInchMin": 1,
    "diagonalInchMax": 999,
    "resolutionHorizontalPixelsMin": 1,
    "resolutionHorizontalPixelsMax": 99999,
    "resolutionVerticalPixelsMin": 1,
    "resolutionVerticalPixelsMax": 99999,
    "sort0": "energyClass",
    "order0": "DESC",
    "sort1": "powerOnModeSDR",
    "order1": "ASC"
}

# Initialize variables for pagination and data collection
all_data = []
seen_ids = set()

while True:
    print(f"Fetching page {params['_page']}...")

    # Send the GET request
    response = requests.get(base_url, headers=headers, params=params)

    if response.status_code == 200:
        # Parse JSON response
        response_json = response.json()

        # Extract records from 'hits'
        data = response_json.get("hits", [])

        # Stop if no more data is available
        if not data:
            print("No more data available. Pagination complete.")
            break

        # Deduplicate entries based on `eprelRegistrationNumber`
        unique_data = [
            item for item in data
            if item.get("eprelRegistrationNumber") not in seen_ids
        ]

        # Update the seen IDs
        seen_ids.update(item.get("eprelRegistrationNumber") for item in unique_data)

        # Append unique records to the list
        all_data.extend(unique_data)

        print(f"Page {params['_page']}: Fetched {len(unique_data)} unique records.")

        # Increment the page for the next iteration
        params["_page"] += 1

        # Respect API rate limits (5 requests per second)
        time.sleep(0.2)

    elif response.status_code == 429:  # Rate-limit handling
        retry_after = int(response.headers.get("Retry-After", 60))
        print(f"Rate limit reached. Retrying after {retry_after} seconds...")
        time.sleep(retry_after)

    else:
        print(f"Error: Request failed with status code {response.status_code}.")
        break

# Convert the collected data to a Pandas DataFrame
df = pd.DataFrame(all_data)

# Save the data to a CSV file
csv_filename = "tumbledriers_data.csv"
df.to_csv(csv_filename, index=False)

print(f"Data successfully saved to {csv_filename}. Total records: {len(df)}")


Fetching page 1...
Page 1: Fetched 100 unique records.
Fetching page 2...
Page 2: Fetched 88 unique records.
Fetching page 3...
Page 3: Fetched 96 unique records.
Fetching page 4...
Page 4: Fetched 100 unique records.
Fetching page 5...
Page 5: Fetched 100 unique records.
Fetching page 6...
Page 6: Fetched 100 unique records.
Fetching page 7...
Page 7: Fetched 100 unique records.
Fetching page 8...
Page 8: Fetched 100 unique records.
Fetching page 9...
Page 9: Fetched 100 unique records.
Fetching page 10...
Page 10: Fetched 100 unique records.
Fetching page 11...
Page 11: Fetched 100 unique records.
Fetching page 12...
Page 12: Fetched 100 unique records.
Fetching page 13...
Page 13: Fetched 100 unique records.
Fetching page 14...
Page 14: Fetched 100 unique records.
Fetching page 15...
Page 15: Fetched 100 unique records.
Fetching page 16...
Page 16: Fetched 100 unique records.
Fetching page 17...
Page 17: Fetched 100 unique records.
Fetching page 18...
Page 18: Fetched 100 unique rec

In [ ]:
import requests
import pandas as pd
import time

# Define the base URL and headers for Light Sources
base_url = "https://eprel.ec.europa.eu/api/products/lightsources"
headers = {
    "x-api-key": "8uDFzzII9X9mzRJ4OLZyk5qZtBKX9JZu5I6szmmN"  # Replace with your valid API key
}

# Filtering and sorting parameters
params = {
    "_page": 1,
    "_limit": 100,  # Fetch up to 100 records per page
    "powerOnModeSDRMin": 0.1,
    "powerOnModeSDRMax": 9999.9,
    "diagonalCmMin": 0.1,
    "diagonalCmMax": 999.9,
    "diagonalInchMin": 1,
    "diagonalInchMax": 999,
    "resolutionHorizontalPixelsMin": 1,
    "resolutionHorizontalPixelsMax": 99999,
    "resolutionVerticalPixelsMin": 1,
    "resolutionVerticalPixelsMax": 99999,
    "sort0": "energyClass",
    "order0": "DESC",
    "sort1": "powerOnModeSDR",
    "order1": "ASC"
}

# Initialize variables for pagination and data collection
all_data = []
seen_ids = set()

while True:
    print(f"Fetching page {params['_page']}...")

    # Send the GET request
    response = requests.get(base_url, headers=headers, params=params)

    if response.status_code == 200:
        # Parse JSON response
        response_json = response.json()

        # Extract records from 'hits'
        data = response_json.get("hits", [])

        # Stop if no more data is available
        if not data:
            print("No more data available. Pagination complete.")
            break

        # Deduplicate entries based on `eprelRegistrationNumber`
        unique_data = [
            item for item in data
            if item.get("eprelRegistrationNumber") not in seen_ids
        ]

        # Update the seen IDs
        seen_ids.update(item.get("eprelRegistrationNumber") for item in unique_data)

        # Append unique records to the list
        all_data.extend(unique_data)

        print(f"Page {params['_page']}: Fetched {len(unique_data)} unique records.")

        # Increment the page for the next iteration
        params["_page"] += 1

        # Respect API rate limits (5 requests per second)
        time.sleep(0.2)

    elif response.status_code == 429:  # Rate-limit handling
        retry_after = int(response.headers.get("Retry-After", 60))
        print(f"Rate limit reached. Retrying after {retry_after} seconds...")
        time.sleep(retry_after)

    else:
        print(f"Error: Request failed with status code {response.status_code}.")
        break

# Convert the collected data to a Pandas DataFrame
df = pd.DataFrame(all_data)

# Save the data to a CSV file
csv_filename = "lights_data.csv"
df.to_csv(csv_filename, index=False)

print(f"Data successfully saved to {csv_filename}. Total records: {len(df)}")


Streaming output truncated to the last 5000 lines.
Page 1701: Fetched 100 unique records.
Fetching page 1702...
Page 1702: Fetched 5 unique records.
Fetching page 1703...
Page 1703: Fetched 100 unique records.
Fetching page 1704...
Page 1704: Fetched 100 unique records.
Fetching page 1705...
Page 1705: Fetched 100 unique records.
Fetching page 1706...
Page 1706: Fetched 100 unique records.
Fetching page 1707...
Page 1707: Fetched 100 unique records.
Fetching page 1708...
Page 1708: Fetched 5 unique records.
Fetching page 1709...
Page 1709: Fetched 100 unique records.
Fetching page 1710...
Page 1710: Fetched 100 unique records.
Fetching page 1711...
Page 1711: Fetched 100 unique records.
Fetching page 1712...
Page 1712: Fetched 100 unique records.
Fetching page 1713...
Page 1713: Fetched 100 unique records.
Fetching page 1714...
Page 1714: Fetched 5 unique records.
Fetching page 1715...
Page 1715: Fetched 100 unique records.
Fetching page 1716...
Page 1716: Fetched 100 unique records.
F

In [ ]:
import requests
import pandas as pd
import time

# Define the base URL and headers for Washer Dryers
base_url = "https://eprel.ec.europa.eu/api/products/washerdriers2019"
headers = {
    "x-api-key": "8uDFzzII9X9mzRJ4OLZyk5qZtBKX9JZu5I6szmmN"  # Replace with your valid API key
}

# Filtering and sorting parameters
params = {
    "_page": 1,
    "_limit": 100,  # Fetch up to 100 records per page
    "powerOnModeSDRMin": 0.1,
    "powerOnModeSDRMax": 9999.9,
    "diagonalCmMin": 0.1,
    "diagonalCmMax": 999.9,
    "diagonalInchMin": 1,
    "diagonalInchMax": 999,
    "resolutionHorizontalPixelsMin": 1,
    "resolutionHorizontalPixelsMax": 99999,
    "resolutionVerticalPixelsMin": 1,
    "resolutionVerticalPixelsMax": 99999,
    "sort0": "energyClass",
    "order0": "DESC",
    "sort1": "powerOnModeSDR",
    "order1": "ASC"
}

# Initialize variables for pagination and data collection
all_data = []
seen_ids = set()

while True:
    print(f"Fetching page {params['_page']}...")

    # Send the GET request
    response = requests.get(base_url, headers=headers, params=params)

    if response.status_code == 200:
        # Parse JSON response
        response_json = response.json()

        # Extract records from 'hits'
        data = response_json.get("hits", [])

        # Stop if no more data is available
        if not data:
            print("No more data available. Pagination complete.")
            break

        # Deduplicate entries based on `eprelRegistrationNumber`
        unique_data = [
            item for item in data
            if item.get("eprelRegistrationNumber") not in seen_ids
        ]

        # Update the seen IDs
        seen_ids.update(item.get("eprelRegistrationNumber") for item in unique_data)

        # Append unique records to the list
        all_data.extend(unique_data)

        print(f"Page {params['_page']}: Fetched {len(unique_data)} unique records.")

        # Increment the page for the next iteration
        params["_page"] += 1

        # Respect API rate limits (5 requests per second)
        time.sleep(0.2)

    elif response.status_code == 429:  # Rate-limit handling
        retry_after = int(response.headers.get("Retry-After", 60))
        print(f"Rate limit reached. Retrying after {retry_after} seconds...")
        time.sleep(retry_after)

    else:
        print(f"Error: Request failed with status code {response.status_code}.")
        break

# Convert the collected data to a Pandas DataFrame
df = pd.DataFrame(all_data)

# Save the data to a CSV file
csv_filename = "washerdriers.csv"
df.to_csv(csv_filename, index=False)

print(f"Data successfully saved to {csv_filename}. Total records: {len(df)}")


Fetching page 1...
Page 1: Fetched 100 unique records.
Fetching page 2...
Page 2: Fetched 100 unique records.
Fetching page 3...
Page 3: Fetched 100 unique records.
Fetching page 4...
Page 4: Fetched 100 unique records.
Fetching page 5...
Page 5: Fetched 100 unique records.
Fetching page 6...
Page 6: Fetched 100 unique records.
Fetching page 7...
Page 7: Fetched 100 unique records.
Fetching page 8...
Page 8: Fetched 100 unique records.
Fetching page 9...
Page 9: Fetched 100 unique records.
Fetching page 10...
Page 10: Fetched 100 unique records.
Fetching page 11...
Page 11: Fetched 100 unique records.
Fetching page 12...
Page 12: Fetched 100 unique records.
Fetching page 13...
Page 13: Fetched 100 unique records.
Fetching page 14...
Page 14: Fetched 100 unique records.
Fetching page 15...
Page 15: Fetched 100 unique records.
Fetching page 16...
Page 16: Fetched 100 unique records.
Fetching page 17...
Page 17: Fetched 100 unique records.
Fetching page 18...
Page 18: Fetched 100 unique r

In [ ]:
import requests
import pandas as pd
import time

# Define the base URL and headers for domestic ovens
base_url = "https://eprel.ec.europa.eu/api/products/ovens"
headers = {
    "x-api-key": "8uDFzzII9X9mzRJ4OLZyk5qZtBKX9JZu5I6szmmN"  # Replace with your valid API key
}

# Initialize variables for pagination
all_data = []  # To store all rows of data
page = 1       # Start from page 1
entries_per_request = 50  # Increase batch size for fewer requests

# Track unique entries
seen_ids = set()  # To track unique entries by `eprelRegistrationNumber`

while True:
    # Construct paginated URL with query parameters
    paginated_url = (
        f"{base_url}?"
        f"_page={page}&_limit={entries_per_request}"
    )

    # Log the current request URL
    print(f"Requesting: {paginated_url}")

    # Make GET request to fetch data
    response = requests.get(paginated_url, headers=headers)

    if response.status_code == 200:
        # Parse JSON response
        response_json = response.json()

        # Extract product data from 'hits'
        data = response_json.get("hits", [])

        # Check if no more data is available
        if not data:
            print("No more data available.")
            break

        # Filter out duplicates by checking unique `eprelRegistrationNumber`
        unique_data = [
            item for item in data
            if item.get("eprelRegistrationNumber") not in seen_ids
        ]

        # Update seen IDs
        seen_ids.update(item.get("eprelRegistrationNumber") for item in unique_data)

        # Append only unique data to the list
        all_data.extend(unique_data)

        print(f"Fetched {len(unique_data)} unique records. Page: {page}")

        # Increment page for the next set of results
        page += 1

        # Add a delay to comply with rate limits (5 requests per second)
        time.sleep(0.5)  # Increase delay to ensure compliance

    elif response.status_code == 429:
        # Handle rate-limiting error by checking Retry-After header or waiting a default time
        retry_after = int(response.headers.get("Retry-After", 60))  # Default to 60 seconds if not provided
        print(f"Rate limit reached. Retrying after {retry_after} seconds...")
        time.sleep(retry_after)

    else:
        print(f"GET request failed with status code {response.status_code}.")
        break

# Convert aggregated data into Pandas DataFrame
df = pd.DataFrame(all_data)

# Save DataFrame to CSV file
csv_filename = "domestic_ovens_data.csv"
df.to_csv(csv_filename, index=False)

print(f"All data successfully saved to {csv_filename}")
